In [513]:
#imports 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns


In [514]:
#reading in dataset train 
df = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('datasets/test.csv')

In [515]:
df_nums = df[['Lot Frontage','Lot Area','Mas Vnr Area','BsmtFin SF 1','BsmtFin SF 2','Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Garage Area']]

## Removing Outlyers 

In [516]:
df.loc[df['Lot Area'] > 100_000, 'Lot Area'] = np.nan

In [517]:
df.loc[df['Total Bsmt SF'] > 5_000, 'Total Bsmt SF'] = np.nan

In [518]:
df.loc[df['1st Flr SF'] > 4_000, '1st Flr SF'] = np.nan

In [519]:
df.loc[df['Lot Frontage'] > 300, 'Lot Frontage'] = np.nan

In [520]:
df.loc[df['Total Bsmt SF'] > 5_000, 'Total Bsmt SF'] = np.nan

In [521]:
df.loc[df['BsmtFin SF 1'] > 3_500, 'BsmtFin SF 1'] = np.nan

In [522]:
df.loc[df['Mas Vnr Area'] > 1_500, 'Mas Vnr Area'] = np.nan

In [523]:
df.loc[df['Total Bsmt SF'] > 4_500, 'Total Bsmt SF'] = np.nan

In [524]:
df.loc[df['Gr Liv Area'] > 4_500, 'Gr Liv Area'] = np.nan

In [525]:
#there seems to be a few very small sale prices, we will remove those 
df.drop(df[df['SalePrice'] <40_000].index, inplace=True)


## Filling NA and getting Dummies

In [526]:
#first lets create the function and some dummies 
def eda_function (df): 
    # replacing NAN in lot with mean to reduce outlyers and still give a data point 
    df['Lot Frontage'].fillna(value=(df['Lot Frontage'].mean()), inplace = True)
    df['Garage Yr Blt'].fillna(value=(df['Garage Yr Blt'].mean()), inplace = True)
    df['BsmtFin SF 1'].fillna(value=(df['BsmtFin SF 1'].mean()), inplace = True)
    df['BsmtFin SF 2'].fillna(value=(df['BsmtFin SF 2'].mean()), inplace = True)
    df['Bsmt Unf SF'].fillna(value=(df['Bsmt Unf SF'].mean()), inplace = True)
    df['Total Bsmt SF'].fillna(value=(df['Total Bsmt SF'].mean()), inplace = True)
    df['Lot Area'].fillna(value=(df['Lot Area'].mean()), inplace = True)
    df['Garage Area'].fillna(value=(df['Garage Area'].mean()), inplace = True)
    df['Garage Yr Blt'].fillna(value=(df['Garage Yr Blt'].mean()), inplace = True)
    df['Garage Cars'].fillna(value=(df['Garage Cars'].mean()), inplace = True)
    df['Gr Liv Area'].fillna(value=(df['Gr Liv Area'].mean()), inplace = True)
    df['1st Flr SF'].fillna(value=(df['1st Flr SF'].mean()), inplace = True)
    #replacing mason area with 0 as it is likely to be the case and it will not create an outlyer
    df['Mas Vnr Area'].fillna(value=(0), inplace = True)

    #making catagories into dummies.  Some of these columns are numerical, but are really catacrorical in nature.  For example, you can not have 3.75 half bathrooms, you can have 3 or 4 half-bedrooms.  
    df = pd.get_dummies(df, columns =['MS SubClass', 'MS Zoning',
       'Street', 'Alley', 'Lot Shape', 'Land Contour',
       'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood',
       'Condition 1', 'Condition 2', 'Bldg Type', 'House Style',
       'Overall Qual', 'Overall Cond', 'Full Bath','Half Bath', 
       'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Bsmt Full Bath',
       'Mas Vnr Type', 'Exter Qual', 'Exter Cond', 'Bsmt Half Bath', 'Bedroom AbvGr',
       'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical','Kitchen Qual', 'Functional', 'Fireplace Qu', 'Garage Type',
       'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Mo Sold', 'Kitchen AbvGr',
       'Pool QC', 'Fence', 'Misc Feature', 'Sale Type'])
    #replacing all other NAs with zero  https://sparkbyexamples.com/pandas/pandas-replace-nan-values-by-zero-in-a-column/#:~:text=Replace%20NaN%20Values%20with%20Zero%20on%20pandas%20DataFrame,but%20returns%20a%20new%20DataFrame.
    #df = df.fillna(0) 
    
    return df

# Sending Data through the cleaning 

In [527]:
df_test = eda_function(df_test) 

In [528]:
df = eda_function(df) 

## Dropping columns that are not in both sets 

In [529]:
#finding those values  https://www.askpython.com/python/list/difference-between-two-lists-unique-entries#:~:text=In%20Python%2C%20to%20find%20the,unique%20entries%20from%20both%20lists.
#taking sale price out so we can add it back in later
sale_price = df['SalePrice']
df = df.drop(columns = list(set(df.columns.values) - set(df_test.columns.values)))
df['SalePrice'] = sale_price

In [530]:
#finding those values  https://www.askpython.com/python/list/difference-between-two-lists-unique-entries#:~:text=In%20Python%2C%20to%20find%20the,unique%20entries%20from%20both%20lists.
df_test= df_test.drop(columns = list(set(df_test.columns.values) - set(df.columns.values)))

## Dropping Non-Corrilated Columns 

In [531]:
corr = (df.corr()[['SalePrice']].sort_values(by = "SalePrice", ascending = False))
corr = corr[92:-75].index
corr = corr.drop('Id', 'SalePrice')
df =df.drop(columns = corr)
df_test = df_test.drop(columns = corr)

##  Exporting Data 

In [532]:
df.to_csv('./datasets/train_cleaned.csv')
df_test.to_csv('datasets/test_cleanded.csv')